In [1]:
from torch.utils.data import DataLoader

from Datasets import BaselineDataset, AllKnobsDataset_train, AllKnobsDataset_test
from Models import TransformerAgent
import torch

In [2]:
# Ensure data directory is populated with https://drive.google.com/drive/folders/1pjHyFlThbpXcvldabLOZZwf4dHaM-p54?usp=share_link

ticker = "AAPL"
data_dir = "../data"
# baseline_dataset = BaselineDataset(ticker, data_dir)
allKnobs_dataset_train = AllKnobsDataset_train(ticker, data_dir)
allKnobs_dataloader_train = DataLoader(allKnobs_dataset_train, batch_size=32, shuffle=True)

allKnobs_dataset_test = AllKnobsDataset_test(ticker, data_dir)
allKnobs_dataloader_test = DataLoader(allKnobs_dataset_test, batch_size=32, shuffle=True)

In [3]:
print(len(allKnobs_dataset_train))
print(len(allKnobs_dataset_test))

1704
365


In [4]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
embedding_dim = len(allKnobs_dataset_train[0][0][0])
hidden_dim = 2048
num_layers = 16
num_atn_heads = 11  # with 22 features, this can only be 1, 2, 11, or 22
checkpoint_dir = "./checkpoints/all_V1"
init_lr = 3
lr_decay = 0.01
min_lr = 0.00000025
decay_lr_every = 2
agent = TransformerAgent(embedding_dim, hidden_dim, num_atn_heads, num_layers, device, checkpoint_dir, init_lr, lr_decay, min_lr, decay_lr_every)

In [ ]:
agent.train(allKnobs_dataloader_train, 500)

Epoch 1, Loss: 16.3689; Average Loss: 0.009606144592130968; lr: [3.0]
Epoch 2, Loss: 12.9594; Average Loss: 0.007605262039482873; lr: [0.03]
Epoch 3, Loss: 13.9144; Average Loss: 0.00816573624109998; lr: [0.03]
Epoch 4, Loss: 13.4796; Average Loss: 0.007910540790904856; lr: [0.00030000000000000003]
Epoch 5, Loss: 13.1021; Average Loss: 0.007689036393949124; lr: [0.00030000000000000003]
Epoch 6, Loss: 13.1621; Average Loss: 0.007724223719656188; lr: [3.0000000000000005e-06]
Epoch 7, Loss: 13.2843; Average Loss: 0.007795940148928356; lr: [3.0000000000000005e-06]
Epoch 8, Loss: 13.0810; Average Loss: 0.007676647000994201; lr: [2.5e-07]
Epoch 9, Loss: 13.1380; Average Loss: 0.007710088359218248; lr: [2.5e-07]
Epoch 10, Loss: 13.2157; Average Loss: 0.007755705903113728; lr: [2.5e-07]
Epoch 11, Loss: 13.0845; Average Loss: 0.007678671342106492; lr: [2.5e-07]
Epoch 12, Loss: 13.0841; Average Loss: 0.0076784630931319205; lr: [2.5e-07]
Epoch 13, Loss: 13.2927; Average Loss: 0.007800887754195733

In [ ]:
# THERE IS a way to use the loss to measure actual "goodness" since it is scaled in an "absolute" way - 0.02 should mean the same "thing" for one prediction as it does for another

In [ ]:
# for (X, y) in allKnobs_dataloader:
#     pred_after = torch.ones(32)
#     actual_after = y[:, 0]
#     actual_prev_day = y[:, 1]
#     
#     metric_true = (actual_after - actual_prev_day) / actual_prev_day
#     metric_pred = (pred_after - actual_prev_day) / actual_prev_day
#     print(actual_prev_day.size())
#     break